In [11]:
import pickle
import pandas as pd
import folium
import geopy
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings("ignore")
import re
import numpy as np
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None  # default='warn'
from IPython.display import clear_output
import geopandas as gpd
from geopy.extra.rate_limiter import RateLimiter
from functools import partial
from geopy import Nominatim

import sys
sys.path.append("../")
import src.supportClean as sc

In [2]:
from geopy import Nominatim
locator = Nominatim(user_agent="myGeocoder")

# data input

In [3]:
df_original = pd.read_json('../data/tripadvisor/test2.json')
print(df_original.shape)
df_original.head()

(10587, 7)


,name_clean,name,price,price_clean,reviews,rating,url
0,NaN,"<a href=""/Restaurant_Review-g187514-d2006562-R...",NaN,"[Cerrado ahora, Española, Internacional, €€ - ...","<span class=""IiChw"">10.747<!-- --> opiniones</...","<span class=""GmcgY""><svg class=""UctUV d H0"" vi...",/Restaurant_Review-g187514-d2006562-Reviews-Lo...
1,NaN,"<a href=""/Restaurant_Review-g187514-d23446200-...",NaN,"[Cerrado ahora, Argentina, Sudamericana]","<span class=""IiChw"">158<!-- --> opiniones</span>","<span class=""GmcgY""><svg class=""UctUV d H0"" vi...",/Restaurant_Review-g187514-d23446200-Reviews-E...
2,NaN,"<a href=""/Restaurant_Review-g187514-d23999961-...",NaN,"[Cerrado ahora, Peruana, Street food / Comida ...","<span class=""IiChw"">305<!-- --> opiniones</span>","<span class=""GmcgY""><svg class=""UctUV d H0"" vi...",/Restaurant_Review-g187514-d23999961-Reviews-L...
3,NaN,"<a href=""/Restaurant_Review-g187514-d18480953-...",NaN,"[Cerrado ahora, Italiana, Pizza, €€ - €€€]","<span class=""IiChw"">720<!-- --> opiniones</span>","<span class=""GmcgY""><svg class=""UctUV d H0"" vi...",/Restaurant_Review-g187514-d18480953-Reviews-P...
4,NaN,"<a href=""/Restaurant_Review-g187514-d12216862-...",NaN,"[Cierra en 10 minutos, Latina, Bar, €€ - €€€]","<span class=""IiChw"">3806<!-- --> opiniones</span>","<span class=""GmcgY""><svg class=""UctUV d H0"" vi...",/Restaurant_Review-g187514-d12216862-Reviews-G...


In [4]:
df_enriched = pd.read_json('../data/tripadvisor/full_test3.json')
print(df_enriched.shape)
df_enriched.head()

(597, 17)


,Name,Link,Province,City,price_all,price,direction,latitude,longitude,cuisines,meals,specialDiets,priceRangeNum,locationAll,coordinates,rating,positionlink
0,Los Montes de Galicia,/Restaurant_Review-g187514-d2006562-Reviews-Lo...,Comunidad de Madrid,Madrid,"[<span class=""DsyBj DxyfE""><a class=""dlMOJ"" hr...",€€ - €€€,"Calle Azcona 46 PISO 8, 28028 Madrid España",NaN,NaN,"Española, Internacional, Mediterránea, Saludable",None,None,50 € - 80 €,NaN,NaN,"[5,0 de 5 burbujas]",NaN
1,Empanadas Malvón,/Restaurant_Review-g187514-d23446200-Reviews-E...,Comunidad de Madrid,Madrid,"[<span class=""DsyBj DxyfE""><a class=""dlMOJ"" hr...",Argentina,"Calle de Luchana 3, 28010 Madrid España",NaN,NaN,"Argentina, Sudamericana","Comidas, Cenas",None,None,NaN,NaN,"[5,0 de 5 burbujas]",NaN
2,La Bajada Street Food,/Restaurant_Review-g187514-d23999961-Reviews-L...,Comunidad de Madrid,Madrid,"[<span class=""DsyBj DxyfE""><a class=""dlMOJ"" hr...",Peruana,Calla Alcalá 414. Ciudad lineal dentro del cen...,NaN,NaN,"Peruana, Street food / Comida en la calle, Sud...","Comidas, Cenas",None,None,NaN,NaN,"[5,0 de 5 burbujas]",NaN
3,Pastamore,/Restaurant_Review-g187514-d18480953-Reviews-P...,Comunidad de Madrid,Madrid,"[<span class=""DsyBj DxyfE""><a class=""dlMOJ"" hr...",€€ - €€€,"Calle Alcalá, 191, 28009 Madrid España",NaN,NaN,"Italiana, Pizza, Mediterránea, Saludable, Tosc...",None,"Opciones vegetarianas, Opciones veganas, Opcio...",15 € - 23 €,NaN,NaN,"[5,0 de 5 burbujas]",NaN
4,Grama Lounge,/Restaurant_Review-g187514-d12216862-Reviews-G...,Comunidad de Madrid,Madrid,"[<span class=""DsyBj DxyfE""><a class=""dlMOJ"" hr...",€€ - €€€,"Calle de la Cruz 19, 28012 Madrid España",NaN,NaN,"Latina, Bar, Venezolana",None,None,5 € - 18 €,NaN,NaN,"[5,0 de 5 burbujas]",NaN


In [5]:
df = df_original.merge(df_enriched, left_on='url', right_on='Link', how = 'right', indicator = True)
df['_merge'].value_counts()

both          597
left_only       0
right_only      0
Name: _merge, dtype: int64

In [6]:
print(df.shape)
df.head()

(597, 25)


,name_clean,name,price_x,price_clean,reviews,rating_x,url,Name,Link,Province,City,price_all,price_y,direction,latitude,longitude,cuisines,meals,specialDiets,priceRangeNum,locationAll,coordinates,rating_y,positionlink,_merge
0,NaN,"<a href=""/Restaurant_Review-g187514-d2006562-R...",NaN,"[Cerrado ahora, Española, Internacional, €€ - ...","<span class=""IiChw"">10.747<!-- --> opiniones</...","<span class=""GmcgY""><svg class=""UctUV d H0"" vi...",/Restaurant_Review-g187514-d2006562-Reviews-Lo...,Los Montes de Galicia,/Restaurant_Review-g187514-d2006562-Reviews-Lo...,Comunidad de Madrid,Madrid,"[<span class=""DsyBj DxyfE""><a class=""dlMOJ"" hr...",€€ - €€€,"Calle Azcona 46 PISO 8, 28028 Madrid España",NaN,NaN,"Española, Internacional, Mediterránea, Saludable",None,None,50 € - 80 €,NaN,NaN,"[5,0 de 5 burbujas]",NaN,both
1,NaN,"<a href=""/Restaurant_Review-g187514-d23446200-...",NaN,"[Cerrado ahora, Argentina, Sudamericana]","<span class=""IiChw"">158<!-- --> opiniones</span>","<span class=""GmcgY""><svg class=""UctUV d H0"" vi...",/Restaurant_Review-g187514-d23446200-Reviews-E...,Empanadas Malvón,/Restaurant_Review-g187514-d23446200-Reviews-E...,Comunidad de Madrid,Madrid,"[<span class=""DsyBj DxyfE""><a class=""dlMOJ"" hr...",Argentina,"Calle de Luchana 3, 28010 Madrid España",NaN,NaN,"Argentina, Sudamericana","Comidas, Cenas",None,None,NaN,NaN,"[5,0 de 5 burbujas]",NaN,both
2,NaN,"<a href=""/Restaurant_Review-g187514-d23999961-...",NaN,"[Cerrado ahora, Peruana, Street food / Comida ...","<span class=""IiChw"">305<!-- --> opiniones</span>","<span class=""GmcgY""><svg class=""UctUV d H0"" vi...",/Restaurant_Review-g187514-d23999961-Reviews-L...,La Bajada Street Food,/Restaurant_Review-g187514-d23999961-Reviews-L...,Comunidad de Madrid,Madrid,"[<span class=""DsyBj DxyfE""><a class=""dlMOJ"" hr...",Peruana,Calla Alcalá 414. Ciudad lineal dentro del cen...,NaN,NaN,"Peruana, Street food / Comida en la calle, Sud...","Comidas, Cenas",None,None,NaN,NaN,"[5,0 de 5 burbujas]",NaN,both
3,NaN,"<a href=""/Restaurant_Review-g187514-d18480953-...",NaN,"[Cerrado ahora, Italiana, Pizza, €€ - €€€]","<span class=""IiChw"">720<!-- --> opiniones</span>","<span class=""GmcgY""><svg class=""UctUV d H0"" vi...",/Restaurant_Review-g187514-d18480953-Reviews-P...,Pastamore,/Restaurant_Review-g187514-d18480953-Reviews-P...,Comunidad de Madrid,Madrid,"[<span class=""DsyBj DxyfE""><a class=""dlMOJ"" hr...",€€ - €€€,"Calle Alcalá, 191, 28009 Madrid España",NaN,NaN,"Italiana, Pizza, Mediterránea, Saludable, Tosc...",None,"Opciones vegetarianas, Opciones veganas, Opcio...",15 € - 23 €,NaN,NaN,"[5,0 de 5 burbujas]",NaN,both
4,NaN,"<a href=""/Restaurant_Review-g187514-d12216862-...",NaN,"[Cierra en 10 minutos, Latina, Bar, €€ - €€€]","<span class=""IiChw"">3806<!-- --> opiniones</span>","<span class=""GmcgY""><svg class=""UctUV d H0"" vi...",/Restaurant_Review-g187514-d12216862-Reviews-G...,Grama Lounge,/Restaurant_Review-g187514-d12216862-Reviews-G...,Comunidad de Madrid,Madrid,"[<span class=""DsyBj DxyfE""><a class=""dlMOJ"" hr...",€€ - €€€,"Calle de la Cruz 19, 28012 Madrid España",NaN,NaN,"Latina, Bar, Venezolana",None,None,5 € - 18 €,NaN,NaN,"[5,0 de 5 burbujas]",NaN,both


# geolocation

In [113]:
df['direction_clean'] = df['direction'].str.lower().str.findall(r'(.*\d{1,3}\s?\W?)\W?.*(\d{5} madrid españa)')
df = df.applymap(lambda x: x if not isinstance(x, list) else x[0] if len(x) else '')
df['direction_clean'].value_counts()

                                                               14
(calle de la cruz 19,, 28012 madrid españa)                     2
(calle echegaray 10,, 28014 madrid españa)                      2
(calle espartinas 7,, 28001 madrid españa)                      2
(calle juan de austria 27,, 28010 madrid españa)                1
                                                               ..
(calle gaztambide 56 gaztambide 56,, 28015 madrid españa)       1
(calle mayor nº 77 calle mayor nº 77,, 28005 madrid españa)     1
(calle de padre damian, 23 , 28036 madrid españa)               1
(calle de martin de los heros 28,, 28008 madrid españa)         1
(calle del gral. pardinas, 70,, 28001 madrid españa)            1
Name: direction_clean, Length: 581, dtype: int64

In [73]:
type(df.applymap(lambda x: x if not isinstance(x, list) else x[0] if len(x) else '').loc[2, 'direction_clean'])

tuple

In [62]:
df_geolocated = sc.geolocation(df, 'direction_clean')

 66%|██████▌   | 392/597 [06:38<07:31,  2.20s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*([('calle de ave maria, 44', '28012 madrid españa')],), **{}).
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/iron2/lib/python3.9/site-packages/urllib3/connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/opt/homebrew/Caskroom/miniforge/base/envs/iron2/lib/python3.9/site-packages/urllib3/connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "/opt/homebrew/Caskroom/miniforge/base/envs/iron2/lib/python3.9/http/client.py", line 1377, in getresponse
    response.begin()
  File "/opt/homebrew/Caskroom/miniforge/base/envs/iron2/lib/python3.9/http/client.py", line 320, in begin
    version, status, reason = self._read_status()
  File "/opt/homebrew/Caskroom/miniforge/base/envs/iron2/lib/python3.9/http/client.py", line 281, in _rea

ValueError: Columns must be same length as key

In [63]:
locator = Nominatim(user_agent="myGeocoder")

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
try:
    df['location'] = df['direction_clean'].progress_apply(geocode)
except:
    clear_output(wait=True)
    pass
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].progress_apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude
#df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
clear_output(wait=True)
df.head(1)

,name_clean,name,price_x,price_clean,reviews,rating_x,url,Name,Link,Province,City,price_all,price_y,direction,latitude,longitude,cuisines,meals,specialDiets,priceRangeNum,locationAll,coordinates,rating_y,positionlink,_merge,direction_clean,location,point
0,NaN,"<a href=""/Restaurant_Review-g187514-d2006562-R...",NaN,"[Cerrado ahora, Española, Internacional, €€ - ...","<span class=""IiChw"">10.747<!-- --> opiniones</...","<span class=""GmcgY""><svg class=""UctUV d H0"" vi...",/Restaurant_Review-g187514-d2006562-Reviews-Lo...,Los Montes de Galicia,/Restaurant_Review-g187514-d2006562-Reviews-Lo...,Comunidad de Madrid,Madrid,"[<span class=""DsyBj DxyfE""><a class=""dlMOJ"" hr...",€€ - €€€,"Calle Azcona 46 PISO 8, 28028 Madrid España",NaN,NaN,"Española, Internacional, Mediterránea, Saludable",None,None,50 € - 80 €,NaN,NaN,"[5,0 de 5 burbujas]",NaN,both,"[(calle azcona 46 piso 8, 28028 madrid españa)]",None,None


In [65]:
print('location null:', df[df['location'].isnull() == True].shape, 'coordinates null:', df[df['point'].isnull() == True].shape)

location null: (146, 28) coordinates null: (146, 28)


In [ ]:
map1 = folium.Map(
    location=[40.41694, -3.70361],
    tiles='cartodbpositron',
    zoom_start=12,
)
df_geolocated[df_geolocated['latitude'].isnull() == False].apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1